In [ ]:
# -----------------------------------------------------------------
# 1. INSTALL REQUIRED LIBRARY
# -----------------------------------------------------------------
# This is needed for the spell-checking function
!pip install fuzzywuzzy

# -----------------------------------------------------------------
# 2. IMPORT LIBRARIES AND LOAD MAIN DATASET
# -----------------------------------------------------------------
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
import pickle
import ast  # For parsing list-like strings from CSV
import warnings # To suppress the UserWarning

# Suppress the specific UserWarning about feature names
warnings.filterwarnings("ignore", message="X does not have valid feature names, but RandomForestClassifier was fitted with feature names")

# loading the dataset
dataset = pd.read_csv('Training.csv')

# printing the dataset
print("Dataset Description:")
print(dataset.describe())

Dataset Description:
           itching    skin_rash  nodal_skin_eruptions  continuous_sneezing  \
count  4920.000000  4920.000000           4920.000000          4920.000000   
mean      0.137805     0.159756              0.021951             0.045122   
std       0.344730     0.366417              0.146539             0.207593   
min       0.000000     0.000000              0.000000             0.000000   
25%       0.000000     0.000000              0.000000             0.000000   
50%       0.000000     0.000000              0.000000             0.000000   
75%       0.000000     0.000000              0.000000             0.000000   
max       1.000000     1.000000              1.000000             1.000000   

         shivering       chills   joint_pain  stomach_pain      acidity  \
count  4920.000000  4920.000000  4920.000000   4920.000000  4920.000000   
mean      0.021951     0.162195     0.139024      0.045122     0.045122   
std       0.146539     0.368667     0.346007      0

In [ ]:
# -----------------------------------------------------------------
# 3. SPLITTING INTO TRAIN AND TEST DATASET
# -----------------------------------------------------------------
X = dataset.drop('prognosis', axis=1)
y = dataset['prognosis']

# encoding prognosis
le = LabelEncoder()
le.fit(y)
Y = le.transform(y)

# Splitting into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=20)

# -----------------------------------------------------------------
# 4. TRAINING THE PREDICTION MODELS
# -----------------------------------------------------------------
# Create a dictionary to store models
prediction_models = {
    'SVC': SVC(kernel='linear'),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
}

for name_of_model, model in prediction_models.items():
    model.fit(X_train, y_train)  # Training the model
    test_predictions = model.predict(X_test)  # Testing the model
    model_accuracy = accuracy_score(y_test, test_predictions)  # Accuracy of the model
    print(f"\n{name_of_model} Accuracy: {model_accuracy} \n")

    # Calculating confusion matrix for all models
    cm = confusion_matrix(y_test, test_predictions)
    print(f"{name_of_model} Confusion Matrix: \n")
    print(np.array2string(cm, separator=', '))


SVC Accuracy: 1.0 

SVC Confusion Matrix: 

[[40,  0,  0, ...,  0,  0,  0],
 [ 0, 43,  0, ...,  0,  0,  0],
 [ 0,  0, 28, ...,  0,  0,  0],
 ...,
 [ 0,  0,  0, ..., 34,  0,  0],
 [ 0,  0,  0, ...,  0, 41,  0],
 [ 0,  0,  0, ...,  0,  0, 31]]

RandomForest Accuracy: 1.0 

RandomForest Confusion Matrix: 

[[40,  0,  0, ...,  0,  0,  0],
 [ 0, 43,  0, ...,  0,  0,  0],
 [ 0,  0, 28, ...,  0,  0,  0],
 ...,
 [ 0,  0,  0, ..., 34,  0,  0],
 [ 0,  0,  0, ...,  0, 41,  0],
 [ 0,  0,  0, ...,  0,  0, 31]]

GradientBoosting Accuracy: 1.0 

GradientBoosting Confusion Matrix: 

[[40,  0,  0, ...,  0,  0,  0],
 [ 0, 43,  0, ...,  0,  0,  0],
 [ 0,  0, 28, ...,  0,  0,  0],
 ...,
 [ 0,  0,  0, ..., 34,  0,  0],
 [ 0,  0,  0, ...,  0, 41,  0],
 [ 0,  0,  0, ...,  0,  0, 31]]


In [ ]:
# -----------------------------------------------------------------
# 5. PRINTING CLASSIFICATION REPORT (USING LAST MODEL'S PREDICTIONS)
# -----------------------------------------------------------------
print("\nClassification Report (for GradientBoosting):")
print(classification_report(y_test, test_predictions))

# -----------------------------------------------------------------
# 6. SELECTING, TRAINING, AND SAVING RANDOM FOREST MODEL
# -----------------------------------------------------------------
# selecting Random Forest Model
Rf = RandomForestClassifier(n_estimators=100, random_state=42)
Rf.fit(X_train, y_train)
ypred = Rf.predict(X_test)
print(f"\nFinal RandomForest Accuracy: {accuracy_score(y_test, ypred)}")

# saving Random Forest
pickle.dump(Rf, open('RandomForest.pkl', 'wb'))

# loading the model
Rf = pickle.load(open('RandomForest.pkl', 'rb'))


Classification Report (for GradientBoosting):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        43
           2       1.00      1.00      1.00        28
           3       1.00      1.00      1.00        46
           4       1.00      1.00      1.00        42
           5       1.00      1.00      1.00        33
           6       1.00      1.00      1.00        33
           7       1.00      1.00      1.00        39
           8       1.00      1.00      1.00        32
           9       1.00      1.00      1.00        49
          10       1.00      1.00      1.00        37
          11       1.00      1.00      1.00        42
          12       1.00      1.00      1.00        41
          13       1.00      1.00      1.00        32
          14       1.00      1.00      1.00        29
          15       1.00      1.00      1.00        30
          16       1.00      1.00 

In [ ]:
# -----------------------------------------------------------------
# 7. TESTING THE PREDICTIONS
# -----------------------------------------------------------------
# first test:
print("\n--- Model Test 1 ---")
prediction_1 = Rf.predict(X_test.iloc[0].values.reshape(1, -1))
print(f"Disease Predicted : {prediction_1}")
print(f"Actual Disease : {y_test[0]}")

# second test:
print("\n--- Model Test 2 ---")
prediction_2 = Rf.predict(X_test.iloc[100].values.reshape(1, -1))
print(f"Disease Predicted : {prediction_2}")
print(f"Actual Disease : {y_test[100]}")

# -----------------------------------------------------------------
# 8. LOADING THE HELPER DATABASES FOR RECOMMENDATIONS
# -----------------------------------------------------------------
# Reading the datasets
symptoms = pd.read_csv("symptoms_df.csv")
precautions = pd.read_csv("precautions_df.csv")
workout = pd.read_csv("workout_df.csv")
description = pd.read_csv("description.csv")
medications = pd.read_csv('medications.csv')
diets = pd.read_csv("diets.csv")


--- Model Test 1 ---
Disease Predicted : [40]
Actual Disease : 40

--- Model Test 2 ---
Disease Predicted : [39]
Actual Disease : 39


In [ ]:
# -----------------------------------------------------------------
# 9. DEFINING HELPER DICTIONARIES AND FUNCTIONS
# -----------------------------------------------------------------

# --- Dictionaries ---
symptoms_list = {'itching': 0, 'skin_rash': 1, 'nodal_skin_eruptions': 2, 'continuous_sneezing': 3, 'shivering': 4, 'chills': 5, 'joint_pain': 6, 'stomach_pain': 7, 'acidity': 8, 'ulcers_on_tongue': 9, 'muscle_wasting': 10, 'vomiting': 11, 'burning_micturition': 12, 'spotting_ urination': 13, 'fatigue': 14, 'weight_gain': 15, 'anxiety': 16, 'cold_hands_and_feets': 17, 'mood_swings': 18, 'weight_loss': 19, 'restlessness': 20, 'lethargy': 21, 'patches_in_throat': 22, 'irregular_sugar_level': 23, 'cough': 24, 'high_fever': 25, 'sunken_eyes': 26, 'breathlessness': 27, 'sweating': 28, 'dehydration': 29, 'indigestion': 30, 'headache': 31, 'yellowish_skin': 32, 'dark_urine': 33, 'nausea': 34, 'loss_of_appetite': 35, 'pain_behind_the_eyes': 36, 'back_pain': 37, 'constipation': 38, 'abdominal_pain': 39, 'diarrhoea': 40, 'mild_fever': 41, 'yellow_urine': 42, 'yellowing_of_eyes': 43, 'acute_liver_failure': 44, 'fluid_overload': 45, 'swelling_of_stomach': 46, 'swelled_lymph_nodes': 47, 'malaise': 48, 'blurred_and_distorted_vision': 49, 'phlegm': 50, 'throat_irritation': 51, 'redness_of_eyes': 52, 'sinus_pressure': 53, 'runny_nose': 54, 'congestion': 55, 'chest_pain': 56, 'weakness_in_limbs': 57, 'fast_heart_rate': 58, 'pain_during_bowel_movements': 59, 'pain_in_anal_region': 60, 'bloody_stool': 61, 'irritation_in_anus': 62, 'neck_pain': 63, 'dizziness': 64, 'cramps': 65, 'bruising': 66, 'obesity': 67, 'swollen_legs': 68, 'swollen_blood_vessels': 69, 'puffy_face_and_eyes': 70, 'enlarged_thyroid': 71, 'brittle_nails': 72, 'swollen_extremeties': 73, 'excessive_hunger': 74, 'extra_marital_contacts': 75, 'drying_and_tingling_lips': 76, 'slurred_speech': 77, 'knee_pain': 78, 'hip_joint_pain': 79, 'muscle_weakness': 80, 'stiff_neck': 81, 'swelling_joints': 82, 'movement_stiffness': 83, 'spinning_movements': 84, 'loss_of_balance': 85, 'unsteadiness': 86, 'weakness_of_one_body_side': 87, 'loss_of_smell': 88, 'bladder_discomfort': 89, 'foul_smell_of urine': 90, 'continuous_feel_of_urine': 91, 'passage_of_gases': 92, 'internal_itching': 93, 'toxic_look_(typhos)': 94, 'depression': 95, 'irritability': 96, 'muscle_pain': 97, 'altered_sensorium': 98, 'red_spots_over_body': 99, 'belly_pain': 100, 'abnormal_menstruation': 101, 'dischromic _patches': 102, 'watering_from_eyes': 103, 'increased_appetite': 104, 'polyuria': 105, 'family_history': 106, 'mucoid_sputum': 107, 'rusty_sputum': 108, 'lack_of_concentration': 109, 'visual_disturbances': 110, 'receiving_blood_transfusion': 111, 'receiving_unsterile_injections': 112, 'coma': 113, 'stomach_bleeding': 114, 'distention_of_abdomen': 115, 'history_of_alcohol_consumption': 116, 'fluid_overload.1': 117, 'blood_in_sputum': 118, 'prominent_veins_on_calf': 119, 'palpitations': 120, 'painful_walking': 121, 'pus_filled_pimples': 122, 'blackheads': 123, 'scurring': 124, 'skin_peeling': 125, 'silver_like_dusting': 126, 'small_dents_in_nails': 127, 'inflammatory_nails': 128, 'blister': 129, 'red_sore_around_nose': 130, 'yellow_crust_ooze': 131}
diseases_list = {15: 'Fungal infection', 4: 'Allergy', 16: 'GERD', 9: 'Chronic cholestasis', 14: 'Drug Reaction', 33: 'Peptic ulcer diseae', 1: 'AIDS', 12: 'Diabetes ', 17: 'Gastroenteritis', 6: 'Bronchial Asthma', 23: 'Hypertension ', 30: 'Migraine', 7: 'Cervical spondylosis', 32: 'Paralysis (brain hemorrhage)', 28: 'Jaundice', 29: 'Malaria', 8: 'Chicken pox', 11: 'Dengue', 37: 'Typhoid', 40: 'hepatitis A', 19: 'Hepatitis B', 20: 'Hepatitis C', 21: 'Hepatitis D', 22: 'Hepatitis E', 3: 'Alcoholic hepatitis', 36: 'Tuberculosis', 10: 'Common Cold', 34: 'Pneumonia', 13: 'Dimorphic hemmorhoids(piles)', 18: 'Heart attack', 39: 'Varicose veins', 26: 'Hypothyroidism', 24: 'Hyperthyroidism', 25: 'Hypoglycemia', 31: 'Osteoarthristis', 5: 'Arthritis', 0: '(vertigo) Paroymsal  Positional Vertigo', 2: 'Acne', 38: 'Urinary tract infection', 35: 'Psoriasis', 27: 'Impetigo'}
symptoms_list_processed = {symptom.replace('_', ' ').lower(): value for symptom, value in symptoms_list.items()}

# --- Information Function ---
def information(predicted_dis):
    disease_description = description[description['Disease'] == predicted_dis]['Description']
    disease_description = " ".join([w for w in disease_description])

    disease_precautions_df = precautions[precautions['Disease'] == predicted_dis]
    disease_precautions = [col for col in disease_precautions_df.values[0][1:]] # Get precautions from the first matched row

    disease_medications_series = medications[medications['Disease'] == predicted_dis]['Medication']
    disease_medications = ast.literal_eval(disease_medications_series.values[0]) if not disease_medications_series.empty else []

    disease_diet_series = diets[diets['Disease'] == predicted_dis]['Diet']
    disease_diet = ast.literal_eval(disease_diet_series.values[0]) if not disease_diet_series.empty else []

    # Changed 'disease' to 'Disease' to match the CSV column header
    disease_workout = workout[workout['Disease'] == predicted_dis]['workout']

    return disease_description, disease_precautions, disease_medications, disease_diet, disease_workout

# --- Prediction Function ---
def predicted_value(patient_symptoms):
    i_vector = np.zeros(len(symptoms_list_processed))
    for i in patient_symptoms:
        if i in symptoms_list_processed:
            i_vector[symptoms_list_processed[i]] = 1
    # Pass the vector in a list, as the model expects a 2D array
    return diseases_list[Rf.predict([i_vector])[0]]

# --- Spell Check Function ---
from fuzzywuzzy import process

def correct_spelling(symptom):
    matched_symptom, score = process.extractOne(symptom, list(symptoms_list_processed.keys()))
    if score >= 80:
        return matched_symptom
    else:
        return None

# -----------------------------------------------------------------
# 10. FINAL TEST - INTERACTIVE PREDICTION
# -----------------------------------------------------------------
print("\n\n--- Interactive Disease Prediction System ---")
symptoms_input = input("Describe your symptoms (separated by commas): ")
patient_symptoms = [s.strip().lower() for s in symptoms_input.split(',')]

corrected_symptoms = []
for symptom in patient_symptoms:
    # *** THIS IS THE FIX ***
    # Changed 'correct_selling' to 'correct_spelling'
    corrected_symptom = correct_spelling(symptom) # Using the spell check function
    if corrected_symptom:
        corrected_symptoms.append(corrected_symptom)
    else:
        print(f"Warning: Symptom '{symptom}' not recognized and will be ignored.")

if corrected_symptoms:
    predicted_disease = predicted_value(corrected_symptoms)
    disease_description, disease_precautions, disease_medications, disease_diet, disease_workout = information(predicted_disease)

    print("\n-------------------------------------------")
    print("Predicted Disease:", predicted_disease)
    print("-------------------------------------------")

    print("\nDescription:")
    print(disease_description)

    print("\nPrecautions:")
    for idx, precaution in enumerate(disease_precautions, start=1):
        if pd.notna(precaution): # Avoid printing 'nan' values
            print(f"{idx}. {precaution.capitalize()}")

    print("\nMedications:")
    for idx, medication in enumerate(disease_medications, start=1):
        print(f"{idx}. {medication}")

    print("\nWorkout:")
    for idx, workout_item in enumerate(disease_workout, start=1):
        print(f"{idx}. {workout_item}")

    print("\nDiets:")
    for idx, diet_item in enumerate(disease_diet, start=1):
        print(f"{idx}. {diet_item}")
else:
    print("\nNo recognized symptoms were provided. Please try again.")



--- Interactive Disease Prediction System ---
Describe your symptoms (separated by commas): stomach_pain, acidity, ulcers_on_tongue, vomiting

-------------------------------------------
Predicted Disease: GERD
-------------------------------------------

Description:
GERD (Gastroesophageal Reflux Disease) is a digestive disorder that affects the lower esophageal sphincter.

Precautions:
1. Gerd
2. Avoid fatty spicy food
3. Avoid lying down after eating
4. Maintain healthy weight
5. Exercise

Medications:
1. Proton Pump Inhibitors (PPIs)
2. H2 Blockers
3. Antacids
4. Prokinetics
5. Antibiotics

Workout:
1. Consume smaller meals
2. Avoid trigger foods (spicy, fatty)
3. Eat high-fiber foods
4. Limit caffeine and alcohol
5. Chew food thoroughly
6. Avoid late-night eating
7. Consume non-citrus fruits
8. Include lean proteins
9. Stay hydrated
10. Avoid carbonated beverages

Diets:
1. Low-Acid Diet
2. Fiber-rich foods
3. Ginger
4. Licorice
5. Aloe vera juice
